**changes to this file were done** (kaggle only)

- incorrect mae calculation

- chosshing the best model

- ...

In [1]:
!pip install -q ultralytics
import os, yaml, tempfile
import pandas as pd
from sklearn.model_selection import KFold
from ultralytics import YOLO
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from shutil import copyfile

# === SETUP ===
label_dir = "/kaggle/input/lacuna-solar-hugojoao/yolo/train/labels"
image_dir = "/kaggle/input/lacuna-solar-hugojoao/yolo/train/images"
all_images = sorted([f for f in os.listdir(image_dir) if f.endswith(".png")])
df = pd.DataFrame({'image': all_images})
k = 5

param_grid = [
    {'cls': 1.5, 'mixup': 0.5, 'copy_paste': 0.5, 'scale': 0.7},
    {'cls': 2.0, 'mixup': 0.3, 'copy_paste': 0.7, 'scale': 0.9},
    {'cls': 2.5, 'mixup': 0.0, 'copy_paste': 0.8, 'scale': 1.0},
]

results = []
kf = KFold(n_splits=k, shuffle=True, random_state=42)

for i, (train_idx, val_idx) in enumerate(kf.split(df)):
    if i >= 3:
        break
    for j, params in enumerate(param_grid):
        print(f"🔁 Fold {i+1}/{k}, Param Set {j+1}: {params}")
        with tempfile.TemporaryDirectory() as fold_dir:
            # Temp structure
            train_img_dir = os.path.join(fold_dir, "train/images")
            val_img_dir = os.path.join(fold_dir, "val/images")
            train_lbl_dir = os.path.join(fold_dir, "train/labels")
            val_lbl_dir = os.path.join(fold_dir, "val/labels")
            os.makedirs(train_img_dir)
            os.makedirs(val_img_dir)
            os.makedirs(train_lbl_dir)
            os.makedirs(val_lbl_dir)

            # Copy train/val data to temp dir
            for idx in train_idx:
                base = df.iloc[idx]["image"]
                copyfile(os.path.join(image_dir, base), os.path.join(train_img_dir, base))
                copyfile(os.path.join(label_dir, base.replace(".png", ".txt")), os.path.join(train_lbl_dir, base.replace(".png", ".txt")))
            for idx in val_idx:
                base = df.iloc[idx]["image"]
                copyfile(os.path.join(image_dir, base), os.path.join(val_img_dir, base))
                copyfile(os.path.join(label_dir, base.replace(".png", ".txt")), os.path.join(val_lbl_dir, base.replace(".png", ".txt")))

            # Create temporary data.yaml
            data_yaml = {
                'train': os.path.join(fold_dir, 'train'),
                'val': os.path.join(fold_dir, 'val'),
                'nc': 2,
                'names': ['thermal', 'photovoltaic']
            }
            yaml_path = os.path.join(fold_dir, 'data.yaml')
            with open(yaml_path, 'w') as f:
                yaml.dump(data_yaml, f)

            # Train
            model = YOLO("yolov8l-seg.pt")
            model.train(
                data=yaml_path,
                project="/kaggle/working",  # Save only logs, not datasets
                name=f"cv_fold_{i}_set_{j}",
                exist_ok=True,
                epochs=50,
                imgsz=512,
                batch=8,
                lr0=0.0001,
                lrf=0.01,
                cls=params['cls'],
                copy_paste=params['copy_paste'],
                mixup=params['mixup'],
                scale=params['scale'],
                amp=True,
                hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
                fliplr=0.5,
                patience=10,
                weight_decay=0.0001,
                box=0.05,
                dfl=1.5,
                momentum=0.937,
                warmup_epochs=5,
                warmup_bias_lr=0.1,
                single_cls=False,
                verbose=False
            )

            torch.cuda.empty_cache()
            metrics = model.val()
            metrics_dict = metrics.results_dict

            # MAE calculations
            def calculate_mae(image_list, label_base, folder_path):
                true_counts, pred_counts = [], []
                for img_name in image_list:
                    label_path = os.path.join(label_base, img_name.replace('.png', '.txt'))
                    true_counts.append(len(open(label_path).readlines()) if os.path.exists(label_path) else 0)
                    results = model(os.path.join(folder_path, img_name))
                    pred_counts.append(len(results[0].boxes))
                return float(np.mean(np.abs(np.array(true_counts) - np.array(pred_counts))))

            val_data = df.iloc[val_idx]
            train_data = df.iloc[train_idx]
            mae_val = calculate_mae(val_data['image'], val_lbl_dir, val_img_dir)
            mae_train = calculate_mae(train_data['image'], train_lbl_dir, train_img_dir)

            results.append({
                'fold': i + 1,
                'set': j + 1,
                'cls': params['cls'],
                'mixup': params['mixup'],
                'copy_paste': params['copy_paste'],
                'scale': params['scale'],
                'mIoU': metrics_dict.get('metrics/mIoU(B)', 0),
                'precision': metrics_dict.get('metrics/precision(B)', 0),
                'recall': metrics_dict.get('metrics/recall(B)', 0),
                'mae_val': mae_val,
                'mae_train': mae_train
            })

# Save and visualize
results_df = pd.DataFrame(results).sort_values(by="mIoU", ascending=False)
results_df.to_csv("/kaggle/working/yolo_imbalance_cv_results.csv", index=False)
print("🏁 DONE — Top Configs:\n", results_df.head())

# Plotting
plt.figure(figsize=(12, 6))
top_n = results_df.head(10)
sns.barplot(data=top_n, x='set', y='mIoU', hue='fold')
plt.title("Top 10 Hyperparameter Combinations by mIoU")
plt.xlabel("Param Set")
plt.ylabel("mIoU")
plt.legend(title="Fold")
plt.tight_layout()
plt.savefig("/kaggle/working/top10_mIoU_barplot.png")
plt.close()

pivot_table = results_df.pivot_table(index='cls', columns='copy_paste', values='mIoU', aggfunc='mean')
plt.figure(figsize=(8, 6))
sns.heatmap(pivot_table, annot=True, fmt=".3f", cmap="YlGnBu")
plt.title("Heatmap: mIoU by cls and copy_paste")
plt.xlabel("copy_paste")
plt.ylabel("cls")
plt.tight_layout()
plt.savefig("/kaggle/working/heatmap_cls_copypaste.png")
plt.close()

# Save best config
best_row = results_df.iloc[0]
best_config = {
    'cls': best_row['cls'],
    'mixup': best_row['mixup'],
    'copy_paste': best_row['copy_paste'],
    'scale': best_row['scale'],
    'batch': 32,
    'lr0': 0.0001
}
with open("/kaggle/working/best_hyperparameters.json", "w") as f:
    json.dump(best_config, f, indent=4)

# Final training
final_model = YOLO("yolov8l-seg.pt")
final_model.train(
    data="/kaggle/input/lacuna-solar-hugojoao/yolo/data.yaml",
    project="/kaggle/working",
    name="final_model_balanced",
    exist_ok=True,
    epochs=75,
    imgsz=512,
    batch=32,
    lr0=0.0001,
    lrf=0.01,
    cls=best_config['cls'],
    mixup=best_config['mixup'],
    copy_paste=best_config['copy_paste'],
    scale=best_config['scale'],
    amp=True,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    fliplr=0.5,
    weight_decay=0.0001,
    box=0.05,
    dfl=1.5,
    momentum=0.937,
    warmup_epochs=5,
    warmup_bias_lr=0.1,
    patience=10,
    single_cls=False
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 24.0 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🔁 Fold 1/5, Param Set 1: {'cls': 1.5, 'mixup': 0.5, 'copy_paste': 0.5, 'scale': 0.7}


100%|██████████| 88.1M/88.1M [00:00<00:00, 111MB/s]


Ultralytics 8.3.105 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=segment, mode=train, model=yolov8l-seg.pt, data=/tmp/tmpy8w0l4mw/data.yaml, epochs=50, time=None, patience=10, batch=8, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=/kaggle/working, name=cv_fold_0_set_0, exist_ok=True, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

100%|██████████| 755k/755k [00:00<00:00, 24.2MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   2360320  ultralytics

...

/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.8ms preprocess, 18.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /kaggle/working/cv_fold_2_set_2
🏁 DONE — Top Configs:
    fold  set  cls  mixup  copy_paste  scale  mIoU  precision    recall  \
0     1    1  1.5    0.5         0.5    0.7     0   0.878007  0.673335   
1     1    2  2.0    0.3         0.7    0.9     0   0.826506  0.707983   
2     1    3  2.5    0.0         0.8    1.0     0   0.845290  0.691627   
3     2    1  1.5    0.5         0.5    0.7     0   0.891651  0.583504   
4     2    2  2.0    0.3         0.7    0.9     0   0.807925  0.602058   

    mae_val  mae_train  
0  0.695266   0.328391  
1  0.751479   0.497405  
2  0.701183   0.418829  
3  1.500000   1.046701  
4  1.547337   0.974796  


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Ultralytics 8.3.105 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


RepresenterError: ('cannot represent an object', 1.5)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import json

# === Save bar plot ===
plt.figure(figsize=(12, 6))
top_n = results_df.sort_values('mIoU', ascending=False).head(10)
sns.barplot(data=top_n, x='set', y='mIoU', hue='fold')
plt.title("Top 10 Hyperparameter Combinations by mIoU")
plt.xlabel("Param Set")
plt.ylabel("mIoU")
plt.legend(title="Fold")
plt.tight_layout()
plt.savefig("/kaggle/working/top10_mIoU_barplot.png")
plt.close()

# === Heatmap: cls vs copy_paste ===
pivot_table = results_df.pivot_table(index='cls', columns='copy_paste', values='mIoU', aggfunc='mean')
plt.figure(figsize=(8, 6))
sns.heatmap(pivot_table, annot=True, fmt=".3f", cmap="YlGnBu")
plt.title("Heatmap: mIoU by cls and copy_paste")
plt.xlabel("copy_paste")
plt.ylabel("cls")
plt.tight_layout()
plt.savefig("/kaggle/working/heatmap_cls_copypaste.png")
plt.close()

# === Save best config to JSON ===
best_row = results_df.iloc[0]
best_config = {
    'cls': best_row['cls'],
    'mixup': best_row['mixup'],
    'copy_paste': best_row['copy_paste'],
    'scale': best_row['scale'],
    'batch': 32,
    'lr0': 0.0001
}
with open("/kaggle/working/best_hyperparameters.json", "w") as f:
    json.dump(best_config, f, indent=4)

# === Optional: Train final model on full dataset ===
# Setup for full training with best config
final_model = YOLO("yolov8l-seg.pt")
final_model.train(
    data="/kaggle/input/lacuna-solar-hugojoao/yolo/data.yaml",
    project="/kaggle/working",
    name="final_model_balanced",
    exist_ok=True,
    epochs=75,
    imgsz=512,
    batch=8,
    lr0=0.0001,
    lrf=0.01,
    cls=best_config['cls'],
    mixup=best_config['mixup'],
    copy_paste=best_config['copy_paste'],
    scale=best_config['scale'],
    amp=True,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    fliplr=0.5,
    weight_decay=0.0001,
    box=0.05,
    dfl=1.5,
    momentum=0.937,
    warmup_epochs=5,
    warmup_bias_lr=0.1,
    patience=10,
    single_cls=False
)


In [ ]:
# === Save class-wise metrics from all folds/sets ===
per_class_records = []

for i, (train_idx, val_idx) in enumerate(kf.split(df)):
    for j, params in enumerate(param_grid):
        run_name = f"cv_fold_{i}_set_{j}"
        model_dir = f"/kaggle/working/{run_name}"
        try:
            model = YOLO(os.path.join(model_dir, "weights/best.pt"))
            metrics = model.val()
            class_metrics = metrics.per_class  # shape: (num_classes, num_metrics)

            for class_id, class_row in enumerate(class_metrics):
                per_class_records.append({
                    'fold': i + 1,
                    'set': j + 1,
                    'class_id': class_id,
                    'class_name': ['thermal', 'photovoltaic'][class_id],
                    'precision': class_row[0],
                    'recall': class_row[1],
                    'mAP50': class_row[2],
                    'mAP50-95': class_row[3],
                    'F1': class_row[4],
                    'IoU': class_row[5]
                })
        except Exception as e:
            print(f"⚠️ Could not load class-wise metrics for {run_name}: {e}")

# === Save to CSV ===
per_class_df = pd.DataFrame(per_class_records)
per_class_df.to_csv("/kaggle/working/classwise_metrics.csv", index=False)

# === Save best config’s class metrics ===
best_model_name = f"cv_fold_{int(best_row['fold']-1)}_set_{int(best_row['set']-1)}"
try:
    best_model = YOLO(f"/kaggle/working/{best_model_name}/weights/best.pt")
    best_metrics = best_model.val().per_class
    best_class_metrics = {
        'thermal': {
            'precision': float(best_metrics[0][0]),
            'recall': float(best_metrics[0][1]),
            'mAP50': float(best_metrics[0][2]),
            'mAP50-95': float(best_metrics[0][3]),
            'F1': float(best_metrics[0][4]),
            'IoU': float(best_metrics[0][5])
        },
        'photovoltaic': {
            'precision': float(best_metrics[1][0]),
            'recall': float(best_metrics[1][1]),
            'mAP50': float(best_metrics[1][2]),
            'mAP50-95': float(best_metrics[1][3]),
            'F1': float(best_metrics[1][4]),
            'IoU': float(best_metrics[1][5])
        }
    }

    with open("/kaggle/working/best_classwise_metrics.json", "w") as f:
        json.dump(best_class_metrics, f, indent=4)

except Exception as e:
    print(f"⚠️ Could not evaluate class-wise metrics for best model: {e}")
